# Delta Lake Lab 
## Unit 9: Table Restore + Storage optimization with OPTIMIZE and VACUUM

In the previous unit we-
1. Learned about table clones

In this unit, we will learn about-
1. Table storage optimization with the OPTIMIZE command
2. Restoring a table to a prior version
2. Table storage optimization with the VACUUM command


### 1. Imports

In [ ]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [ ]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

### 3. Declare variables

In [ ]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

In [ ]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

In [ ]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

In [ ]:
ACCOUNT_NAME = "YOUR_ACCOUNT_NAME"

In [ ]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

### 4. File listing

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

### 5. Optimize 
Delta Lake can improve the speed of read queries from a table by coalescing small files into larger ones through the "OPTIMIZE TABLE" command.
It is important to schedule periodic runs of this command for performance in a busy environment with too many small files.

Note: Optimize does not remove the small files, they still need to be deleted. It merely creates larger files that are indexed.

- Bin-packing optimization is idempotent, meaning that if it is run twice on the same dataset, the second run has no effect.

- Bin-packing aims to produce evenly-balanced data files with respect to their size on disk, but not necessarily number of tuples per file. However, the two measures are most often correlated.

- Python and Scala APIs for executing OPTIMIZE operation are available from Delta Lake 2.0 and above.

- Set Spark session configuration spark.databricks.delta.optimize.repartition.enabled=true to use repartition(1) instead of coalesce(1) for better performance when compacting many small files.

In [ ]:
spark.sql("OPTIMIZE "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta").show(truncate=False)

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

Note that the parquet file count has not reduced. 

### 6. Restore to a prior version of the table

In [ ]:
spark.sql("RESTORE TABLE "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta TO VERSION AS OF 5").show(truncate=False)

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* 

### 7. Vacuum - for deleting small files
Vacuum purges the small files that are not indexed.

In [ ]:
spark.sql("VACUUM "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta").show(truncate=False)

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
spark.sql("VACUUM "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta RETAIN 0 HOURS")

Delta does not allow you to VACUUM with 0 retention unless you have the below property set-

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled" , False)

Now VACUUM should work..

In [ ]:
spark.sql("VACUUM "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta RETAIN 0 HOURS")

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

### THIS CONCLUDES THIS LAB. DONT FORGET TO SHUT DOWN THE LAB RESOURCES.